In [ ]:
!gdown --id 1iQ93IWVdR6dZ6W7RahbLq166u-6ADelJ

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1iQ93IWVdR6dZ6W7RahbLq166u-6ADelJ
From (redirected): https://drive.google.com/uc?id=1iQ93IWVdR6dZ6W7RahbLq166u-6ADelJ&confirm=t&uuid=22f1bcf6-da8f-4cca-9633-e8e054d2d4e2
To: /content/data.zip
100% 2.34G/2.34G [00:22<00:00, 104MB/s] 


In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import math
from PIL import Image, ImageDraw
from PIL import ImagePath
import pandas as pd

from os import path
from tqdm import tqdm
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
import urllib
%tensorflow_version 2.x
import tensorflow as tf

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
!unzip -qn data.zip

In [ ]:
image_dir='data/images'
import os

In [ ]:
image_list=[]
for i in sorted(os.listdir(image_dir)):
  # print(i)
  folder_dir=os.path.join(image_dir,i)
  for image in sorted(os.listdir(folder_dir)):
    image_directory=os.path.join(folder_dir,image)
    image_list.append(image_directory)

In [ ]:
mask_dir='data/mask'

In [ ]:
mask_list=[]
for i in sorted(os.listdir(mask_dir)):
  # print(i)
  folder_dir=os.path.join(mask_dir,i)
  for mask in sorted(os.listdir(folder_dir)):
    mask_directory=os.path.join(folder_dir,mask)
    mask_list.append(mask_directory)

In [ ]:
import pandas as pd
data_df= pd.DataFrame(columns=['image','json'])

In [ ]:
data_df['image']=image_list
data_df['json']=mask_list

In [ ]:
data_df.shape

(4008, 2)

In [ ]:
import json
from tqdm.auto import tqdm
a=[dir]
def return_unique_labels(data_df):
    # for each file in the column json
    # read and store all the objects present in that file
    # compute the unique objects and retrun them
    # if open any json file using any editor you will get better sense of it
    for i in tqdm(data_df['json']):
      f = open(i,)
      data =json.load(f)
      for j in data['objects']:
        a.append((j.get('label')))
    return a
    # return unique_labels

In [ ]:
unique_labels = return_unique_labels(data_df)

  0%|          | 0/4008 [00:00<?, ?it/s]

In [ ]:
def unique(list1):
    list_set = set(list1)
    unique_list = (list(list_set))
    return unique_list
unique_labels=unique(unique_labels)

In [ ]:
len(unique_labels)

41

In [ ]:
(unique_labels)

['out of roi',
 'road',
 'traffic sign',
 'bridge',
 'motorcycle',
 'bicycle',
 'sidewalk',
 'trailer',
 'building',
 'ground',
 'rectification border',
 'obs-str-bar-fallback',
 'pole',
 'rail track',
 'drivable fallback',
 'truck',
 'sky',
 'billboard',
 'non-drivable fallback',
 'bus',
 'unlabeled',
 'rider',
 'wall',
 'curb',
 <function dir>,
 'train',
 'traffic light',
 'polegroup',
 'guard rail',
 'ego vehicle',
 'parking',
 'vegetation',
 'vehicle fallback',
 'fallback background',
 'tunnel',
 'animal',
 'fence',
 'person',
 'caravan',
 'autorickshaw',
 'car']

In [ ]:
unique_labels[26]

'train'

In [ ]:
unique_labels.remove(unique_labels[26])

In [ ]:
label_clr = {'road':10, 'parking':20, 'drivable fallback':20,'sidewalk':30,'non-drivable fallback':40,'rail track':40,\
                        'person':50, 'animal':50, 'rider':60, 'motorcycle':70, 'bicycle':70, 'autorickshaw':80,\
                        'car':80, 'truck':90, 'bus':90, 'vehicle fallback':90, 'trailer':90, 'caravan':90,\
                        'curb':100, 'wall':100, 'fence':110,'guard rail':110, 'billboard':120,'traffic sign':120,\
                        'traffic light':120, 'pole':130, 'polegroup':130, 'obs-str-bar-fallback':130,'building':140,\
                        'bridge':140,'tunnel':140, 'vegetation':150, 'sky':160, 'fallback background':160,'unlabeled':0,\

            'out of roi':0, 'ego vehicle':170, 'ground':180,'rectification border':190,\
                   'train':200}

In [ ]:
def grader_2(unique_labels):
    if (not (set(label_clr.keys())-set(unique_labels))) and len(unique_labels) == 40:
        print("True")
    else:
        print("False")

grader_2(unique_labels)

False


In [ ]:
def get_poly(file):
    f = open(file,)
    data = json.load(f)
    label,vertexlist=[],[]
    for obj in data['objects']:
        label.append(obj['label'])
        vertexlist.append([tuple(vertex) for vertex in obj['polygon']])
    w= data['imgWidth']
    h=data['imgHeight']
    return w, h, label, vertexlist

In [1]:
import matplotlib.pyplot as plt
import math
from PIL import Image, ImageDraw
from PIL import ImagePath
import numpy as np


In [ ]:
def grader_3(file):
    w, h, labels, vertexlist = get_poly(file)
    print(len((set(labels)))==18 and len(vertexlist)==227 and w==1920 and h==1080 \
          and isinstance(vertexlist,list) and isinstance(vertexlist[0],list) and isinstance(vertexlist[0][0],tuple) )

grader_3('data/mask/201/frame0029_gtFine_polygons.json')

True


In [ ]:
def compute_masks(data_df):
    mask=[]
    for file in tqdm(data_df['json']):
        w, h, labels, vertexlist = get_poly(file)

        img= Image.new("RGB",(w,h))
        img1 = ImageDraw.Draw(img)
        for i in range(len(labels)):
            if(len(vertexlist[i])>1):
                img1.polygon(vertexlist[i], fill = label_clr[labels[i]])
        img=np.array(img)
        im = Image.fromarray(img[:,:,0])
        new_file=file.replace('mask','output')
        new_file=new_file.replace('json','png')
        os.makedirs('data/output/'+file.split('/')[2],exist_ok=True)
        im.save(new_file)
        mask.append(new_file)
    data_df['mask']=mask

    return data_df

In [ ]:
data_df = compute_masks(data_df)
data_df.head()

  0%|          | 0/4008 [00:00<?, ?it/s]

,image,json,mask
0,data/images/201/frame0029_leftImg8bit.jpg,data/mask/201/frame0029_gtFine_polygons.json,data/output/201/frame0029_gtFine_polygons.png
1,data/images/201/frame0299_leftImg8bit.jpg,data/mask/201/frame0299_gtFine_polygons.json,data/output/201/frame0299_gtFine_polygons.png
2,data/images/201/frame0779_leftImg8bit.jpg,data/mask/201/frame0779_gtFine_polygons.json,data/output/201/frame0779_gtFine_polygons.png
3,data/images/201/frame1019_leftImg8bit.jpg,data/mask/201/frame1019_gtFine_polygons.json,data/output/201/frame1019_gtFine_polygons.png
4,data/images/201/frame1469_leftImg8bit.jpg,data/mask/201/frame1469_gtFine_polygons.json,data/output/201/frame1469_gtFine_polygons.png


In [ ]:
data_df.to_csv('/content/drive/MyDrive/data_df.csv',index=False)

In [ ]:
import joblib
joblib.dump(data_df,'data.pkl')

['data.pkl']

In [ ]:
import joblib
data_df = joblib.load('data.pkl')

In [ ]:
import urllib.request

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf

In [ ]:
def conv2d_block(input_tensor, n_filters, kernel_size = 3):
  x = input_tensor
  for i in range(2):
    x = tf.keras.layers.Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
            kernel_initializer = 'he_normal', padding = 'same')(x)
    x = tf.keras.layers.Activation('relu')(x)
  return x
def encoder_block(inputs, n_filters=64, pool_size=(2,2), dropout=0.3):
  f = conv2d_block(inputs, n_filters=n_filters)
  p = tf.keras.layers.MaxPooling2D(pool_size=pool_size)(f)
  p = tf.keras.layers.Dropout(0.3)(p)
  return f, p
def encoder(inputs):
  f1, p1 = encoder_block(inputs, n_filters=64, pool_size=(2,2), dropout=0.3)
  f2, p2 = encoder_block(p1, n_filters=128, pool_size=(2,2), dropout=0.3)
  f3, p3 = encoder_block(p2, n_filters=256, pool_size=(2,2), dropout=0.3)
  f4, p4 = encoder_block(p3, n_filters=512, pool_size=(2,2), dropout=0.3)
  return p4, (f1, f2, f3, f4)

In [ ]:
def bottleneck(inputs):
  bottle_neck = conv2d_block(inputs, n_filters=1024)
  return bottle_neck

In [ ]:
# Decoder Utilities

def decoder_block(inputs, conv_output, n_filters=64, kernel_size=3, strides=3, dropout=0.3):

  u = tf.keras.layers.Conv2DTranspose(n_filters, kernel_size, strides = strides, padding = 'same')(inputs)
  c = tf.keras.layers.concatenate([u, conv_output])
  c = tf.keras.layers.Dropout(dropout)(c)
  c = conv2d_block(c, n_filters, kernel_size=3)

  return c


def decoder(inputs, convs, output_channels):


  f1, f2, f3, f4 = convs

  c6 = decoder_block(inputs, f4, n_filters=512, kernel_size=(3,3), strides=(2,2), dropout=0.3)
  c7 = decoder_block(c6, f3, n_filters=256, kernel_size=(3,3), strides=(2,2), dropout=0.3)
  c8 = decoder_block(c7, f2, n_filters=128, kernel_size=(3,3), strides=(2,2), dropout=0.3)
  c9 = decoder_block(c8, f1, n_filters=64, kernel_size=(3,3), strides=(2,2), dropout=0.3)

  outputs = tf.keras.layers.Conv2D(output_channels, (1, 1), activation='softmax')(c9)
  #outputs=tf.keras.layers.Flatten()(outputs)
  #outputs=tf.squeeze(outputs)

  return outputs



In [ ]:
OUTPUT_CHANNELS = 21

def unet():

  inputs = tf.keras.layers.Input(shape=(256,256,3))
  encoder_output, convs = encoder(inputs)
  bottle_neck = bottleneck(encoder_output)
  outputs = decoder(bottle_neck, convs, output_channels=OUTPUT_CHANNELS)
  model = tf.keras.Model(inputs=inputs, outputs=outputs)
  return model
model = unet()
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d_19 (Conv2D)          (None, 256, 256, 64)         1792      ['input_2[0][0]']             
                                                                                                  
 activation_18 (Activation)  (None, 256, 256, 64)         0         ['conv2d_19[0][0]']           
                                                                                                  
 conv2d_20 (Conv2D)          (None, 256, 256, 64)         36928     ['activation_18[0][0]']       
                                                                                            

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test = train_test_split(data_df, test_size=0.11, random_state=42)

In [ ]:
import imgaug.augmenters as iaa
# For the assignment choose any 4 augumentation techniques
# check the imgaug documentations for more augmentations
aug2 = iaa.Fliplr(1)
aug3 = iaa.Flipud(1)
aug4 = iaa.Emboss(alpha=(1), strength=1)
aug5 = iaa.DirectedEdgeDetect(alpha=(0.8), direction=(1.0))
aug6 = iaa.Sharpen(alpha=(1.0), lightness=(1.5))

In [ ]:
def visualize(**images):
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        if i==1:
            plt.imshow(image, cmap='gray', vmax=1, vmin=0)
        else:
            plt.imshow(image)
    plt.show()


class Dataset:
    # we will be modifying this CLASSES according to your data/problems
    # CLASSES = a

    # the parameters needs to changed based on your requirements
    # here we are collecting the file_names because in our dataset, both our images and maks will have same file name
    # ex: fil_name.jpg   file_name.mask.jpg
    def __init__(self , basepath , images_file_names , mask_file_names , CLASSES):

        self.image_ids = images_file_names
        self.mask_ids = mask_file_names
        # the paths of images
        self.images_fps   = [os.path.join(basepath,image_id) for image_id in self.image_ids]

        # the paths of segmentation images
        self.masks_fps    = [os.path.join(basepath,mask_id) for mask_id in self.mask_ids]

        # giving labels for each class
        self.CLASSES= CLASSES
        self.class_values = CLASSES

    def __getitem__(self, i):

        # read data
        image = cv2.imread(self.images_fps[i], cv2.IMREAD_UNCHANGED)
        image = cv2.resize(image,(256,256),interpolation=cv2.INTER_NEAREST)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


        mask  = cv2.imread(self.masks_fps[i], cv2.IMREAD_UNCHANGED)
        image_mask = cv2.resize(mask,(256,256))
        image_masks = [(image_mask == v) for v in self.class_values]
        image_mask = np.stack(image_masks, axis=-1).astype('float')

        if self.images_fps[i] in x_train:
          a = np.random.uniform()
          if a<0.2:
              image = aug2.augment_image(image)
              image_mask = aug2.augment_image(image_mask)
          elif a<0.4:
              image = aug3.augment_image(image)
              image_mask = aug3.augment_image(image_mask)
          elif a<0.6:
              image = aug4.augment_image(image)
              image_mask = aug4.augment_image(image_mask)
          elif a<0.8:
              image = aug5.augment_image(image)
              image_mask = image_mask
          else:
              image = aug6.augment_image(image)
              image_mask = aug6.augment_image(image_mask)

        return image, image_mask

    def __len__(self):
        return len(self.image_ids)


class Dataloder(tf.keras.utils.Sequence):
    def __init__(self, dataset, batch_size=1, shuffle=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(dataset))

    def __getitem__(self, i):

        # collect batch data
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])

        batch = [np.stack(samples, axis=0) for samples in zip(*data)]

        return tuple(batch)

    def __len__(self):
        return len(self.indexes) // self.batch_size

    def on_epoch_end(self):
        if self.shuffle:
            self.indexes = np.random.permutation(self.indexes)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
CLASSES = list(np.unique(list(label_clr.values())))

In [ ]:
print(len(CLASSES))

21


In [ ]:
from keras.callbacks import Callback, ModelCheckpoint,TensorBoard,ReduceLROnPlateau
from keras.callbacks import TensorBoard
import time
# import datetime
!rm -rf '/content/logs'
CLASSES = CLASSES
basepath = '/content/'
train_dataset = Dataset(basepath, x_train['image'].values, x_train['mask'].values, CLASSES=CLASSES)
test_dataset = Dataset(basepath, x_test['image'].values, x_test['mask'].values, CLASSES=CLASSES)
train_dataloader = Dataloder(train_dataset, batch_size=16, shuffle=True)
test_dataloader = Dataloder(test_dataset, batch_size=16, shuffle=True)
print(train_dataloader[0][0].shape)
print(train_dataloader[0][1].shape)
# assert train_dataloader[0][0].shape == (BATCH_SIZE, 256, 256, 3)
# assert train_dataloader[0][1].shape == (BATCH_SIZE, 256, 256, 21)
file_path='/content/drive/MyDrive/my_model2.h5'
tensorboard = TensorBoard(log_dir="/content/logs/{}".format(time.time()), histogram_freq=1, write_graph=True,write_grads=True)
checkpoint = ModelCheckpoint(filepath=file_path, save_weights_only=True,save_best_only=True, mode='max', monitor='val_accuracy')
red = ReduceLROnPlateau(monitor='val_iou_score', min_lr=0.000001,patience=2)
call = [tensorboard]

(16, 256, 256, 3)


(16, 256, 256, 21)


In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
history = model.fit(train_dataloader, steps_per_epoch=len(train_dataloader), epochs=4,\
                              validation_data=test_dataloader,callbacks=call)

Epoch 1/4
222/222 [==============================] - 375s 1s/step - loss: 4.4722 - accuracy: 0.5146 - val_loss: 1.1847 - val_accuracy: 0.6439
Epoch 2/4
222/222 [==============================] - 319s 1s/step - loss: 1.0668 - accuracy: 0.6715 - val_loss: 0.9171 - val_accuracy: 0.7114
Epoch 3/4
222/222 [==============================] - 313s 1s/step - loss: 0.9116 - accuracy: 0.7125 - val_loss: 0.8697 - val_accuracy: 0.7136
Epoch 4/4
222/222 [==============================] - 310s 1s/step - loss: 0.8464 - accuracy: 0.7260 - val_loss: 0.8000 - val_accuracy: 0.7373


In [ ]:
weights_path = '/content/drive/MyDrive/u_net_weights1.h5'
model_path='/content/drive/MyDrive/u_net.h5'
model.save_weights(weights_path)
model.save(model_path)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model.load_weights(weights_path)

In [ ]:
a=0
for i in range(len(x_test)):
    a+=1
    #original image
    image = cv2.imread(list(x_test['image'])[i], cv2.IMREAD_UNCHANGED)
    image = cv2.resize(image, (256,256),interpolation = cv2.INTER_NEAREST)

    #predicted segmentation map
    pred_mask  = model.predict(image[np.newaxis,:,:,:])
    pred_mask = tf.argmax(pred_mask, axis=-1)

    #original segmentation map
    image_mask = cv2.imread(list(x_test['mask'])[i], cv2.IMREAD_UNCHANGED)
    image_mask = cv2.resize(image_mask, (256,256),interpolation = cv2.INTER_NEAREST)


    plt.figure(figsize=(10,6))
    plt.subplot(131)
    plt.imshow(image)
    plt.subplot(132)
    plt.imshow(image_mask)
    plt.subplot(133)
    plt.imshow(pred_mask[0])
    plt.show()
    if a==20:
      break

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import cv2
import os

SAVE_PATH = os.path.join(os.getcwd(),'drive','MyDrive')
# Read the video from specified path
cam = cv2.VideoCapture(os.path.join(SAVE_PATH,'VIDEO.mp4'))

try:

	# creating a folder named data
	if not os.path.exists('video'):
		os.makedirs('video')

# if not created then raise error
except OSError:
	print ('Error: Creating directory of video')

# frame
currentframe = 0

while(True):

	# reading from frame
    ret,frame = cam.read()

    if ret:
        # if video is still left continue creating images
        name = os.path.join(os.getcwd(),'video','frame' + str(currentframe) + '.jpg')
        #name = './data/frame' + str(currentframe) + '.jpg'
        print ('Creating...' + name)

        # writing the extracted images
        cv2.imwrite(name, frame)

        # increasing counter so that it will
        # show how many frames are created
        currentframe += 1
    else:
      break

# Release all space and windows once done
cam.release()
cv2.destroyAllWindows()

Streaming output truncated to the last 5000 lines.
Creating.../content/video/frame140.jpg
Creating.../content/video/frame141.jpg
Creating.../content/video/frame142.jpg
Creating.../content/video/frame143.jpg
Creating.../content/video/frame144.jpg
Creating.../content/video/frame145.jpg
Creating.../content/video/frame146.jpg
Creating.../content/video/frame147.jpg
Creating.../content/video/frame148.jpg
Creating.../content/video/frame149.jpg
Creating.../content/video/frame150.jpg
Creating.../content/video/frame151.jpg
Creating.../content/video/frame152.jpg
Creating.../content/video/frame153.jpg
Creating.../content/video/frame154.jpg
Creating.../content/video/frame155.jpg
Creating.../content/video/frame156.jpg
Creating.../content/video/frame157.jpg
Creating.../content/video/frame158.jpg
Creating.../content/video/frame159.jpg
Creating.../content/video/frame160.jpg
Creating.../content/video/frame161.jpg
Creating.../content/video/frame162.jpg
Creating.../content/video/frame163.jpg
Creating.../c

In [ ]:
'''Creating and saving our masks which the model predicted for the video'''
i=0
if(not os.path.exists(os.path.join(os.getcwd(),'drive','MyDrive','video_mask'))):
  os.mkdir(os.path.join(os.getcwd(),'drive','MyDrive','video_mask'))

frames = len(os.listdir(os.path.join(os.getcwd(),'video')))
frames

#iterating over all the video frames
while(i!=frames):

  #Getting the image abd resizing it according to our input model
  image_path = os.path.join(os.getcwd(),'video','frame'+str(i)+'.jpg')
  image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
  image = cv2.resize(image,(256,256),interpolation=cv2.INTER_AREA)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

  #getting the prediction mask image
  prediction = model.predict(image[np.newaxis,:,:,:])
  prediction = tf.argmax(prediction,axis=-1)
  prediction = tf.expand_dims(prediction,axis=-1)

  #saving the mask image
  plt.figure(figsize=(10,6))
  plt.subplot(132)
  plt.imshow(prediction[0,:,:,0])
  mask_path = os.path.join(os.getcwd(),'drive','MyDrive','video_mask','frame'+str(i)+'.png')
  print('Creating...',mask_path)
  plt.savefig(mask_path)
  plt.close()
  i+=1

1/1 [==============================] - 0s 33ms/step
Creating... /content/drive/MyDrive/video_mask/frame0.png
1/1 [==============================] - 0s 35ms/step
Creating... /content/drive/MyDrive/video_mask/frame1.png
1/1 [==============================] - 0s 35ms/step
Creating... /content/drive/MyDrive/video_mask/frame2.png
1/1 [==============================] - 0s 33ms/step
Creating... /content/drive/MyDrive/video_mask/frame3.png
1/1 [==============================] - 0s 36ms/step
Creating... /content/drive/MyDrive/video_mask/frame4.png
1/1 [==============================] - 0s 30ms/step
Creating... /content/drive/MyDrive/video_mask/frame5.png
1/1 [==============================] - 0s 25ms/step
Creating... /content/drive/MyDrive/video_mask/frame6.png
1/1 [==============================] - 0s 19ms/step
Creating... /content/drive/MyDrive/video_mask/frame7.png
1/1 [==============================] - 0s 20ms/step
Creating... /content/drive/MyDrive/video_mask/frame8.png
1/1 [==============

KeyboardInterrupt: 